In [4]:
# Importing Libraries
import selenium
import pandas as pd
import time
from bs4 import BeautifulSoup

#!pip install webdriver-manager

# Importing selenium webdriver 
from selenium import webdriver 
from webdriver_manager.chrome import ChromeDriverManager

#connect to the webdriver
driver = webdriver.Chrome(ChromeDriverManager().install())


# Importing required Exceptions which needs to handled
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException

#Importing requests
import requests

# importing regex
import re



====== WebDriver manager ======
Current google-chrome version is 94.0.4606
Get LATEST driver version for 94.0.4606
There is no [win32] chromedriver for browser 94.0.4606 in cache
Get LATEST driver version for 94.0.4606
Trying to download new driver from https://chromedriver.storage.googleapis.com/94.0.4606.61/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\Csp\.wdm\drivers\chromedriver\win32\94.0.4606.61]


In [5]:
#Setting ticket types paths
return_ticket = "//label[@id='flight-type-roundtrip-label-hp-flight']"
one_way_ticket = "//label[@id='flight-type-one-way-label-hp-flight']"
multi_ticket = "//label[@id='flight-type-multi-dest-label-hp-flight']"

In [6]:
def ticket_chooser(ticket):

    try:
        ticket_type = browser.find_element_by_xpath(ticket)
        ticket_type.click()
    except Exception as e:
        pass

In [7]:
def dep_country_chooser(dep_country):
    fly_from = browser.find_element_by_xpath("//input[@id='flight-origin-hp-flight']")
    time.sleep(1)
    fly_from.clear()
    time.sleep(1.5)
    fly_from.send_keys('  ' + dep_country)
    time.sleep(1.5)
    first_item = browser.find_element_by_xpath("//a[@id='aria-option-0']")
    time.sleep(1.5)
    first_item.click()

In [8]:
def arrival_country_chooser(arrival_country):
    fly_to = browser.find_element_by_xpath("//input[@id='flight-destination-hp-flight']")
    time.sleep(1)
    fly_to.clear()
    time.sleep(1.5)
    fly_to.send_keys('  ' + arrival_country)
    time.sleep(1.5)
    first_item = browser.find_element_by_xpath("//a[@id='aria-option-0']")
    time.sleep(1.5)
    first_item.click()

In [9]:
def dep_date_chooser(month, day, year):

    dep_date_button = browser.find_element_by_xpath("//input[@id='flight-departing-hp-flight']")
    dep_date_button.clear()
    dep_date_button.send_keys(month + '/' + day + '/' + year)


In [10]:
def return_date_chooser(month, day, year):
    return_date_button = browser.find_element_by_xpath("//input[@id='flight-returning-hp-flight']")

    for i in range(11):
        return_date_button.send_keys(Keys.BACKSPACE)
    return_date_button.send_keys(month + '/' + day + '/' + year)


In [11]:
def search():
    search = browser.find_element_by_xpath("//button[@class='btn-primary btn-action gcw-submit']")
    search.click()
    time.sleep(15)
    print('Results ready!')


In [14]:
df = pd.DataFrame()
def compile_data():
    global df
    global dep_times_list
    global arr_times_list
    global airlines_list
    global price_list
    global durations_list
    global stops_list
    global layovers_list

    #departure times
    dep_times = browser.find_elements_by_xpath("//span[@data-test-id='departure-time']")
    dep_times_list = [value.text for value in dep_times]

    #arrival times
    arr_times = browser.find_elements_by_xpath("//span[@data-test-id='arrival-time']")
    arr_times_list = [value.text for value in arr_times]
    
    # prices
    prices = browser.find_elements_by_xpath("//span[@data-test-id='listing-price-dollars']")
    price_list = [value.text for value in prices]

    #airline name
    airlines = browser.find_elements_by_xpath("//span[@data-test-id='airline-name']")
    airlines_list = [value.text for value in airlines]


    #durations
    durations = browser.find_elements_by_xpath("//span[@data-test-id='duration']")
    durations_list = [value.text for value in durations]

    #stops
    stops = browser.find_elements_by_xpath("//span[@class='number-stops']")
    stops_list = [value.text for value in stops]

    #layovers
    layovers = browser.find_elements_by_xpath("//span[@data-test-id='layover-airport-stops']")
    layovers_list = [value.text for value in layovers]

    now = datetime.datetime.now()
    current_date = (str(now.year) + '-' + str(now.month) + '-' + str(now.day))
    current_time = (str(now.hour) + ':' + str(now.minute))
    current_price = 'price' + '(' + current_date + '---' + current_time + ')'
    for i in range(len(dep_times_list)):
        try:
            df.loc[i, 'departure_time'] = dep_times_list[i]
        except Exception as e:
            pass
        try:
            df.loc[i, 'arrival_time'] = arr_times_list[i]
        except Exception as e:
            pass
        try:
            df.loc[i, 'airline'] = airlines_list[i]
        except Exception as e:
            pass
        try:
            df.loc[i, 'duration'] = durations_list[i]
        except Exception as e:
            pass
        try:
            df.loc[i, 'stops'] = stops_list[i]
        except Exception as e:
            pass
        try:
            df.loc[i, 'layovers'] = layovers_list[i]
        except Exception as e:
            pass
        try:
            df.loc[i, str(current_price)] = price_list[i]
        except Exception as e:
            pass

    print('Excel Sheet Created!')

In [15]:
now = datetime.datetime.now()
current_date = (str(now.year) + '-' + str(now.month) + '-' + str(now.day))
current_time = (str(now.hour) + ':' + str(now.minute))
current_price = 'price' + '(' + current_date + '---' + current_time + ')'

In [16]:
def GetPathForExcelsOutPut(journeyDetails,i):
    dep_arr_name = journeyDetails.at[i,'dep_country_chooser'] + "_" + journeyDetails.at[i,'arrival_country_chooser']
    sampleTime   = "sampleTime_" + str(pd.to_datetime('today').strftime("%d/%m/%Y")).replace("/", "_")
    dep_date     = str("%02d" % journeyDetails.at[i,'dep_month']) + "_" + str("%02d" % journeyDetails.at[i,'dep_day']) + "_" + str(journeyDetails.at[i,'dep_year'])
    arr_date     = str("%02d" % journeyDetails.at[i,'arr_month']) + "_" + str("%02d" % journeyDetails.at[i,'arr_day']) + "_" + str(journeyDetails.at[i,'arr_year'])
    conc_date    = "depDate_" + dep_date + "_arrDate_" + arr_date 
    data_path = "Insert your data results folder here (the Parent folder that contain the all automatic subdir that will be created)"
    folderPath   = os.path.join(data_path,dep_arr_name, conc_date, 'sampleTime\\')
    return [folderPath, conc_date]
 
def SaveDfToCsv(journeyDetails,i):
    checkPathExist(journeyDetails,i)
    [pathForDepArrDate, nameOfFolder] = GetPathForExcelsOutPut(journeyDetails,i)
    df['departure_date'] = GetDepartDateUsingFolderName(nameOfFolder) # send file path to function
    df['arrival_date'] = GetReturnDateUsingFolderName(nameOfFolder) # send file path to function
    df.to_csv(str(GetPathForExcelsOutPut(journeyDetails,i)[0]) + "_" + dt.datetime.today().strftime('%y%m%d-%H%M%S')+ ".csv", index = False)